# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

<span style="color:#107896", max-width="105ex">
This is a binary classification problem, since the prediction the system is required to do is of categorical kind, out of two mutually exclusive outputs, that is: "student passes" and "student fails"
</span>

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [45]:
# Import libraries
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt


pd.set_option('display.width', 500)

In [46]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [84]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1] - 1
n_passed = student_data.passed.value_counts()['yes']
n_failed = student_data.passed.value_counts()['no']
grad_rate = np.float(n_passed)/n_students*100
print "\nTotal number of students:      {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features:            {}".format(n_features)
print "Graduation rate of the class:  {:.2f}%".format(grad_rate)


Total number of students:      395
Number of students who passed: 265
Number of students who failed: 130
Number of features:            30
Graduation rate of the class:  67.09%


#### A more detailed view of available data through a Data Quality Report

Let's define a Python class that collects more detailed central tendency information about the available data

In [48]:
class DataQualityReport(object):    
    '''
    Report about data basic statistics
    '''
    
    CONT_TABLE_COLS = ['count', 'missing', 'cardinality', 'min', '1st_quartile', 
                       'mean', 'median', '3rd_quartile', 'max', 'std_dev']
    CAT_TABLE_COLS  = ['count', 'missing', 'cardinality', 'mode', 'mode_freq', 
                       'mode_percent', '2nd_mode', '2nd_mode_freq', '2nd_mode_percent']
        
    def __init__(self, df, continuous_features, categorical_features):
        '''
        Constructor
        Parameters:
            df - pandas DataFrame object containing the dataset
            categorical_features - the list of feature names of categorical type
            continuous_features - the list of feature names of continuous type
        '''
        self._continuous_features   = continuous_features
        self._categorical_features  = categorical_features

        self.cont_table = pd.DataFrame(columns=DataQualityReport.CONT_TABLE_COLS, index=continuous_features)
        self.cat_table  = pd.DataFrame(columns=DataQualityReport.CAT_TABLE_COLS, index=categorical_features)
        stats = df.describe()
        self._populate_cont_table(df, stats)
        self._populate_cat_table(df, stats)
  
    def _populate_cont_table(self, df, stats):
        for feature in self._continuous_features:
            self.cont_table['count'][feature]           = df[feature].value_counts().sum()
            self.cont_table['missing'][feature]         = df[feature].isnull().sum()
            self.cont_table['cardinality'][feature]     = df[feature].unique().shape[0]
            self.cont_table['min'][feature]             = stats[feature]['min']
            self.cont_table['1st_quartile'][feature]    = stats[feature]['25%']
            self.cont_table['mean'][feature]            = stats[feature]['mean']
            self.cont_table['median'][feature]          = stats[feature]['50%']
            self.cont_table['3rd_quartile'][feature]    = stats[feature]['75%']
            self.cont_table['max'][feature]             = stats[feature]['max']
            self.cont_table['std_dev'][feature]         = stats[feature]['std']
            
            
    def _populate_cat_table(self, df, stats):
        for feature in self._categorical_features:
            self.cat_table['count'][feature]           = df[feature].value_counts().sum()
            self.cat_table['missing'][feature]         = df[feature].isnull().sum()
            self.cat_table['cardinality'][feature]     = df[feature].unique().shape[0]
            vc = df[feature].value_counts()
            self.cat_table['mode'][feature]            = vc.index[0]
            self.cat_table['mode_freq'][feature]       = vc.values[0]
            self.cat_table['mode_percent'][feature]    = float(vc.values[0])/vc.sum()*100
            if vc.shape[0] > 1:
                self.cat_table['2nd_mode'][feature]            = vc.index[1]
                self.cat_table['2nd_mode_freq'][feature]       = vc.values[1]
                self.cat_table['2nd_mode_percent'][feature]    = float(vc.values[1])/vc.sum()*100

In our dataset, we find the following continuous/numeric descriptive features:

In [49]:
numeric_features = ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 
                    'goout', 'Dalc', 'Walc', 'health', 'absences']

and the following categorical features:

In [50]:
categorical_features = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 
                        'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher','internet', 'romantic']

Let's build a data quality report:

In [51]:
dqr = DataQualityReport(student_data, numeric_features, categorical_features)

print data quality report for numeric features:

In [82]:
print "\nNUMERIC FEATURES DATA QUALITY REPORT:"
dqr.cont_table


NUMERIC FEATURES DATA QUALITY REPORT:


,count,missing,cardinality,min,1st_quartile,mean,median,3rd_quartile,max,std_dev
age,395,0,8,15,16,16.6962,17,18,22,1.27604
Medu,395,0,5,0,2,2.74937,3,4,4,1.09474
Fedu,395,0,5,0,2,2.52152,2,3,4,1.0882
traveltime,395,0,4,1,1,1.4481,1,2,4,0.697505
studytime,395,0,4,1,1,2.03544,2,2,4,0.83924
failures,395,0,4,0,0,0.334177,0,0,3,0.743651
famrel,395,0,5,1,4,3.9443,4,5,5,0.896659
freetime,395,0,5,1,3,3.23544,3,4,5,0.998862
goout,395,0,5,1,2,3.10886,3,4,5,1.11328
Dalc,395,0,5,1,1,1.48101,1,2,5,0.890741


and print data quality report table for categorical features:

In [83]:
print "\nCATEGORICAL FEATURES DATA QUALITY REPORT:"
dqr.cat_table


CATEGORICAL FEATURES DATA QUALITY REPORT:


,count,missing,cardinality,mode,mode_freq,mode_percent,2nd_mode,2nd_mode_freq,2nd_mode_percent
school,395,0,2,GP,349,88.3544,MS,46,11.6456
sex,395,0,2,F,208,52.6582,M,187,47.3418
address,395,0,2,U,307,77.7215,R,88,22.2785
famsize,395,0,2,GT3,281,71.1392,LE3,114,28.8608
Pstatus,395,0,2,T,354,89.6203,A,41,10.3797
Mjob,395,0,5,other,141,35.6962,services,103,26.0759
Fjob,395,0,5,other,217,54.9367,services,111,28.1013
reason,395,0,4,course,145,36.7089,home,109,27.5949
guardian,395,0,3,mother,273,69.1139,father,90,22.7848
schoolsup,395,0,2,no,344,87.0886,yes,51,12.9114


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [71]:
# Extract feature (X) and target (y) columns
from IPython.display import display, HTML

feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print
print "Target column: {}".format(target_col)

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed


In [78]:
X_all = student_data[feature_cols]  # feature values for all students
print "\nFeature values:-"
display(X_all.head())  # print the first 5 rows



Feature values:-


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,yes,no,no,4,3,4,1,1,3,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,yes,no,5,3,3,1,1,3,4
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,yes,no,4,3,2,2,3,3,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,yes,3,2,2,1,1,5,2
4,GP,F,16,U,GT3,T,3,3,other,other,...,yes,no,no,4,3,2,1,2,5,4


In [79]:
print "\nTarget values:-"
y_all = student_data[target_col]  # corresponding targets/labels
y_all.head()


Target values:-


0     no
1     no
2    yes
3    yes
4    yes
Name: passed, dtype: object

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [55]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [56]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn import cross_validation as cv

test_size=np.float(num_test)/num_all
X_train, X_test, y_train, y_test = cv.train_test_split(X_all, y_all, test_size=test_size, random_state=42)
#X_train = ?
#y_train = ?
#X_test = ?
#y_test = ?
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

#### Define a class to collect performance measures for each experiment

In [57]:
class PerformanceTable(object):
    '''
    Collect measures of performance for a specific experiment
    '''

    MEASURES = ["TRAIN_TIME", "PREDICT_TIME_TRAIN", "PREDICT_TIME_TEST", "F1SCORE_TRAIN", "F1SCORE_TEST"]
    
    def __init__(self, training_set_sizes):
        '''
        Constructor
        '''
        self.table = pd.DataFrame(columns=PerformanceTable.MEASURES, index=training_set_sizes)

In [58]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    #print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    training_time = end - start
    #print "Done!\nTraining time (secs): {:.3f}".format(training_time)
    return training_time

# TODO: Choose a model, import it and instantiate an object
from sklearn.tree import DecisionTreeClassifier 
clf = DecisionTreeClassifier(min_samples_leaf=5,min_weight_fraction_leaf=0.02, max_depth=3, class_weight='balanced')

# Fit model to training data
#train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it
#print clf

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.024
DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.02,
            presort=False, random_state=None, splitter='best')


In [60]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    #print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    predict_time = end - start
    #print "Done!\nPrediction time (secs): {:.3f}".format(predict_time)
#    print "Predictions: {}, {}".format(y_pred[:5], type(y_pred))
#    print "Targets: {}, {}".format(target[:5], type(target))
    return f1_score(target.astype(str), y_pred.astype(str), pos_label='yes'), predict_time

train_f1_score,_ = predict_labels(clf, X_train, y_train)
#print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.834101382488


In [61]:
# Predict on test data
#print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: (0.82014388489208612, 0.0002460479736328125)


In [64]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    training_time = train_classifier(clf, X_train, y_train)
    F1Score_train, predict_time_train = predict_labels(clf, X_train, y_train)
    print "F1 score for training set: {}".format(F1Score_train)
    F1Score_test, predict_time_test = predict_labels(clf, X_test, y_test)
    print "F1 score for test set: {}".format(F1Score_test)
    return training_time, F1Score_train, predict_time_train, F1Score_test, predict_time_test

# TODO: Run the helper function above for desired subsets of training data
training_set_sizes = range(20,301,20)
perf = PerformanceTable(training_set_sizes)
for train_set_size in training_set_sizes:
    #print "Using training set size {}:".format(train_set_size)
    training_time, F1Score_train, predict_time_train, F1Score_test, predict_time_test = train_predict(clf, 
                                                X_train[:train_set_size],y_train[:train_set_size], X_test, y_test)
    perf.table['TRAIN_TIME'][train_set_size] = training_time*1000 # milliseconds
    perf.table['PREDICT_TIME_TRAIN'][train_set_size] = predict_time_train*1000 # milliseconds
    perf.table['PREDICT_TIME_TEST'][train_set_size] = predict_time_test*1000 # milliseconds
    perf.table['F1SCORE_TRAIN'][train_set_size] = F1Score_train
    perf.table['F1SCORE_TEST'][train_set_size] = F1Score_test
    
print "Perfomance table for DecisionTreeClassifier: "
print perf.table
# Note: Keep the test set constant

------------------------------------------
Training set size: 20
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.909090909091
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.677419354839
------------------------------------------
Training set size: 40
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.8
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.660869565217
------------------------------------------
Training set size: 60
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for traini

In [11]:
# TODO: Train and predict using two other models

# Second model
from sklearn.naive_bayes import GaussianNB 
clf = GaussianNB()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it
print clf

Training GaussianNB...
Done!
Training time (secs): 0.002
GaussianNB()


In [12]:
train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.036
F1 score for training set: 0.80378250591


In [13]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.763358778626


In [14]:
# Run the helper function above for desired subsets of training data
for train_set_size in range(50,301,25):
    #print "Using training set size {}:".format(train_set_size)
    train_predict(clf, X_train[:train_set_size], y_train[:train_set_size], X_test, y_test)
# Note: Keep the test set constant

------------------------------------------
Training set size: 50
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.612244897959
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.58
------------------------------------------
Training set size: 75
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.86
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.763358778626
------------------------------------------
Training set size: 100
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.846715328467
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 scor

In [19]:
# Third model
from sklearn.linear_model import LogisticRegression 
clf = LogisticRegression(penalty='l2',class_weight="balanced", max_iter=1000)

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it
print clf

Training LogisticRegression...
Done!
Training time (secs): 0.053
LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=1000,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)


In [22]:
train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.782383419689


In [23]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.756302521008


In [25]:
# Run the helper function above for desired subsets of training data
for train_set_size in range(50,301,25):
    #print "Using training set size {}:".format(train_set_size)
    train_predict(clf, X_train[:train_set_size], y_train[:train_set_size], X_test, y_test)
# Note: Keep the test set constant

------------------------------------------
Training set size: 50
Training LogisticRegression...
Done!
Training time (secs): 0.003
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.866666666667
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.666666666667
------------------------------------------
Training set size: 75
Training LogisticRegression...
Done!
Training time (secs): 0.004
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.914893617021
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.721804511278
------------------------------------------
Training set size: 100
Training LogisticRegression...
Done!
Training time (secs): 0.004
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.8292682926

In [42]:
# model 4
from sklearn.linear_model import SGDClassifier 
clf = SGDClassifier(shuffle=True, class_weight='balanced')

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it
print clf

Training SGDClassifier...
Done!
Training time (secs): 0.002
SGDClassifier(alpha=0.0001, average=False, class_weight='balanced',
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)


In [43]:
train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SGDClassifier...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.811158798283


In [44]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using SGDClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.788732394366


In [45]:
# Run the helper function above for desired subsets of training data
for train_set_size in range(50,301,25):
    #print "Using training set size {}:".format(train_set_size)
    train_predict(clf, X_train[:train_set_size], y_train[:train_set_size], X_test, y_test)
# Note: Keep the test set constant

------------------------------------------
Training set size: 50
Training SGDClassifier...
Done!
Training time (secs): 0.001
Predicting labels using SGDClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.805555555556
Predicting labels using SGDClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.746268656716
------------------------------------------
Training set size: 75
Training SGDClassifier...
Done!
Training time (secs): 0.001
Predicting labels using SGDClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.8
Predicting labels using SGDClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.774193548387
------------------------------------------
Training set size: 100
Training SGDClassifier...
Done!
Training time (secs): 0.001
Predicting labels using SGDClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.530612244898
Predicting labels using SGDClassifier...
Done!
P

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [ ]:
# TODO: Fine-tune your model and report the best F1 score